In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
import scanpy as sc

In [3]:
adata = sc.read_h5ad("/home/augusta/SSS_mount/insituCNV/data/WTx-CosMx_TVA/WTx-CosMx/adata_1970164_processed_metadata_processed_logcountCNV_epi.h5ad")

In [4]:
adata

AnnData object with n_obs × n_vars = 387605 × 18555
    obs: 'Unnamed: 0', 'fov', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'Width', 'Height', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD68_CK8_18', 'Max.CD68_CK8_18', 'Mean.CD298_B2M', 'Max.CD298_B2M', 'Mean.CD45', 'Max.CD45', 'Mean.DAPI', 'Max.DAPI', 'cell_id', 'version', 'Run_name', 'Run_Tissue_name', 'tissue', 'Panel', 'assay_type', 'slide_ID', 'CenterX_global_px', 'CenterY_global_px', 'cell_ID', 'unassignedTranscripts', 'nCount_RNA', 'nFeature_RNA', 'nCount_negprobes', 'nFeature_negprobes', 'nCount_falsecode', 'nFeature_falsecode', 'Area.um2', 'cell', 'sid', 'add_cell_id', 'add_ist', 'add_sub', 'n_counts', 'leiden', 'cnv_leiden_res0.2', 'cnv_leiden_res0.5'
    var: 'Unnamed: 0', 'start', 'end', 'width', 'names', 'gene_id', 'symbol', 'ensg', 'chromosome'
    uns: 'add_ist_colors', 'add_sub_colors', 'cnv', 'cnv_leiden_res0.2', 'cnv_leiden_res0.2_colors', 'cnv_leiden_res0.5', 'cnv_leiden_res0.5_colors', 'cnv_neighbors', 'd

In [6]:
adata.uns

{'add_ist_colors': array(['#6a3a4c', '#1b4400', '#4fc601', '#3b5dff', '#4a3b53', '#ff2f80',
        '#61615a', '#ba0900', '#6b7900', '#00c2a0', '#ffaa92', '#ff90c9',
        '#b903aa', '#c2ff99'], dtype=object),
 'add_sub_colors': array(['#1f77b4'], dtype=object),
 'cnv': {'chr_pos': {'chr1': 0,
   'chr10': 798,
   'chr11': 858,
   'chr12': 972,
   'chr13': 1060,
   'chr14': 1081,
   'chr15': 1130,
   'chr16': 1175,
   'chr17': 1243,
   'chr18': 1344,
   'chr19': 1360,
   'chr2': 184,
   'chr20': 1486,
   'chr21': 1528,
   'chr22': 1539,
   'chr3': 293,
   'chr4': 386,
   'chr5': 448,
   'chr6': 518,
   'chr7': 607,
   'chr8': 682,
   'chr9': 736}},
 'cnv_leiden_res0.2': {'params': {'n_iterations': 2,
   'random_state': 0,
   'resolution': 0.2}},
 'cnv_leiden_res0.2_colors': array(['#1f77b4', '#ff7f0e', '#279e68'], dtype=object),
 'cnv_leiden_res0.5': {'params': {'n_iterations': 2,
   'random_state': 0,
   'resolution': 0.5}},
 'cnv_leiden_res0.5_colors': array(['#1f77b4', '#ff7f0e', '

### sankey

In [30]:
import pandas as pd
import plotly.graph_objects as go

def plot_sankey(adata, category1, category2, title="Sankey Plot"):
    """
    Create a Sankey plot to show the correspondence between two categories in adata.obs.

    Parameters:
    - adata (AnnData): An AnnData object with observation categories to compare.
    - category1 (str): The name of the first category in adata.obs (e.g., "cnv_leiden").
    - category2 (str): The name of the second category in adata.obs (e.g., "add_ist").
    - title (str): The title of the Sankey plot.

    Returns:
    - fig (plotly.graph_objects.Figure): The Sankey plot.
    """
    
    # Get counts for combinations of the two categories with observed=True to avoid FutureWarning
    data = adata.obs.groupby([category1, category2], observed=True).size().reset_index(name='count')

    # Convert category columns to string type to avoid TypeErrors in indexing
    data[category1] = data[category1].astype(str)
    data[category2] = data[category2].astype(str)
    
    # Create label lists for nodes
    labels1 = list(data[category1].unique())
    labels2 = list(data[category2].unique())
    labels = labels1 + labels2  # Combine labels for all nodes
    
    # Create mappings from labels to node indices
    label_to_index = {label: i for i, label in enumerate(labels)}
    
    # Map categories to node indices and counts to link values
    source = data[category1].map(label_to_index).values
    target = data[category2].map(label_to_index).values + len(labels1)  # Shift index for second category
    value = data['count'].values

    # Define the Sankey plot data
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color="blue"
        ),
        link=dict(
            source=source,
            target=target,
            value=value
        )
    ))

    # Update layout
    fig.update_layout(
        title_text=title,
        font_size=10
    )
    
    return fig

In [10]:
import pandas as pd
import plotly.graph_objects as go

def plot_sankey(adata, category1, category2, title="Sankey Plot"):
    """
    Create a Sankey plot to show the correspondence between two categories in adata.obs.

    Parameters:
    - adata (AnnData): An AnnData object with observation categories to compare.
    - category1 (str): The name of the first category in adata.obs (e.g., "cnv_leiden_res0.2").
    - category2 (str): The name of the second category in adata.obs (e.g., "add_ist").
    - title (str): The title of the Sankey plot.

    Returns:
    - fig (plotly.graph_objects.Figure): The Sankey plot.
    """
    
    # Get counts for combinations of the two categories
    data = adata.obs.groupby([category1, category2], observed=True).size().reset_index(name='count')

    # Convert category columns to string type to avoid TypeErrors in indexing
    data[category1] = data[category1].astype(str)
    data[category2] = data[category2].astype(str)
    
    # Create label lists for nodes
    labels1 = list(data[category1].unique())
    labels2 = list(data[category2].unique())
    labels = labels1 + labels2  # Combine labels for all nodes
    
    # Create mappings from labels to node indices
    label_to_index = {label: i for i, label in enumerate(labels)}
    
    # Retrieve colors for each label based on categories
    cnv_colors = adata.uns['cnv_leiden_res0.2_colors'][:len(labels1)]
    ist_colors = adata.uns['add_ist_colors'][:len(labels2)]
    node_colors = cnv_colors + ist_colors  # Combine colors for each node
    
    # Map categories to node indices and counts to link values
    source = data[category1].map(label_to_index).values
    target = data[category2].map(label_to_index).values + len(labels1)  # Shift index for second category
    value = data['count'].values
    
    # Define the Sankey plot data
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color=node_colors  # Apply predefined colors to each node
        ),
        link=dict(
            source=source,
            target=target,
            value=value
        )
    ))

    # Update layout
    fig.update_layout(
        title_text=title,
        font_size=10
    )
    
    return fig

# Example usage
sankey_fig = plot_sankey(adata, category1='cnv_leiden_res0.2', category2='add_ist', title="CNV Clusters vs Phenotypic Clusters")
sankey_fig.show()


ValueError: operands could not be broadcast together with shapes (3,) (14,) 

In [31]:
# Example usage: Generate a Sankey plot for 'cnv_leiden' and 'add_ist' categories
sankey_fig = plot_sankey(adata, category1='cnv_leiden_res0.2', category2='add_ist', title="CNV Clusters vs Phenotypic Clusters")
sankey_fig.show()

In [9]:
sankey_fig.write_html("sankey_plot_ist_epi_2.html")

In [37]:
print(set(adata.obs.add_ist.unique()))

{'epi. secret. progen.', 'epi. entero. 2', 'epi. entero. 1', 'epi. stem 1', 'epi. stem 2', 'fib.', 'epi. secret.', 'epi. goblet', 'epi. absorp. 1', 'epi. stem 3', 'unresolved', 'epi. absorp. 3', 'epi. absorp. 2', 'epi. progen.'}


### Circle plot

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

def plot_circle_distribution(adata, category1, category2, title="CNV Cluster - IST Distribution"):
    """
    Create a series of circle plots (pie charts) to show the distribution of category2 (e.g., IST clusters) 
    within each level of category1 (e.g., CNV clusters).

    Parameters:
    - adata (AnnData): An AnnData object with observation categories to compare.
    - category1 (str): The name of the first category in adata.obs (e.g., "cnv_leiden").
    - category2 (str): The name of the second category in adata.obs (e.g., "add_ist").
    - title (str): The title of the circle distribution plots.

    Returns:
    - fig (plotly.graph_objects.Figure): The circle distribution plot with subplots for each CNV cluster.
    """
    
    # Get counts for combinations of the two categories
    data = adata.obs.groupby([category1, category2], observed=True).size().reset_index(name='count')
    
    # Convert category columns to string type
    data[category1] = data[category1].astype(str)
    data[category2] = data[category2].astype(str)
    
    # Get unique CNV clusters
    unique_clusters = data[category1].unique()
    
    # Create a subplot grid for each CNV cluster, specifying type "domain" for pie charts
    rows = len(unique_clusters)
    fig = sp.make_subplots(
        rows=rows, cols=1, 
        subplot_titles=[f"CNV cluster {cluster}" for cluster in unique_clusters],
        specs=[[{"type": "domain"}] for _ in range(rows)]
    )
    
    for i, cluster in enumerate(unique_clusters, start=1):
        # Filter data for the current CNV cluster
        cluster_data = data[data[category1] == cluster]
        
        # Create a pie chart for the distribution of IST clusters within the current CNV cluster
        pie_fig = px.pie(cluster_data, names=category2, values='count', hole=0.3)
        
        # Update the hover template to show percentage and set IST cluster as the main label
        pie_fig.update_traces(hovertemplate="%{label}: %{percent}", textinfo="label")
        
        # Add the pie chart to the subplot
        fig.add_trace(pie_fig.data[0], row=i, col=1)
    
    # Update layout for overall figure
    fig.update_layout(
        title_text=title,
        height=300 * rows,
        showlegend=False
    )
    
    return fig

In [20]:
# Example usage
category1 = 'cnv_leiden_res0.5'
category2='leiden'
circle_fig = plot_circle_distribution(adata, category1=category1, category2=category2, title="CNV Clusters - IST Cluster Distribution")

In [21]:
circle_fig.write_html(f'circleplot_{category1}_{category2}.html')